## Import packages and data

In [1]:
# import packages
import pandas as pd
import numpy as np
import os 
import re

from random import sample
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

import m2cgen as m2c

import warnings
warnings.filterwarnings("ignore")

In [2]:
# import data
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [3]:
# assume that there are missing values in the first 2 columns of the data
sequence = [i for i in range(len(X))]
subset0 = sample(sequence, 30)
subset1 = sample(sequence, 50)
subset2 = sample(sequence, 40)
subset3 = sample(sequence, 60)
X[[(subset0)],0] = np.nan
X[[(subset1)],1] = np.nan
X[[(subset0)],2] = np.nan
X[[(subset1)],3] = np.nan

In [4]:
pd.DataFrame(X).isna().sum()

0    30
1    50
2    30
3    50
dtype: int64

## Train a simple model

In [5]:
# split data into train and test sets
seed = 2020
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [6]:
print(pd.DataFrame(X_train).isna().sum())
print(pd.DataFrame(X_test).isna().sum())

0    22
1    36
2    22
3    36
dtype: int64
0     8
1    14
2     8
3    14
dtype: int64


In [7]:
# fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

[00:21:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Convert XGBoost model to VBA

In [8]:
code = m2c.export_to_visual_basic(model, function_name = 'pred')

In [9]:
print(code)

Module Model
Function pred(ByRef inputVector() As Double) As Double()
    Dim var0 As Double
    If (inputVector(2)) < (2.45) Then
        var0 = 0.41652894
    Else
        If (inputVector(3)) < (0.6) Then
            var0 = 0.257142842
        Else
            var0 = -0.209508225
        End If
    End If
    Dim var1 As Double
    If (inputVector(2)) < (2.45) Then
        var1 = 0.289331138
    Else
        If (inputVector(3)) < (0.6) Then
            var1 = 0.211369321
        Else
            var1 = -0.18690227
        End If
    End If
    Dim var2 As Double
    If (inputVector(2)) < (2.45) Then
        var2 = 0.231908903
    Else
        If (inputVector(3)) < (0.6) Then
            var2 = 0.176793814
        Else
            var2 = -0.170180246
        End If
    End If
    Dim var3 As Double
    If (inputVector(2)) < (2.45) Then
        var3 = 0.199418262
    Else
        If (inputVector(3)) < (0.6) Then
            var3 = 0.15227066
        Else
            var3 = -0.157188073

## Manual Scripts to convert VBA to SAS

In [10]:
# remove unnecessary things
code = re.sub('Dim var.* As Double', '', code)
code = re.sub('End If', '', code)

# change the beginning
code = re.sub('Module Model\nFunction pred\(ByRef inputVector\(\) As Double\) As Double\(\)\n', 
                'DATA pred_result;\nSET dataset_name;', code)

# change the ending
code = re.sub('End Function\nEnd Module\n', 'RUN;', code)

# insert ';'
all_match_list = re.findall('[0-9]+\n', code)
for idx in range(len(all_match_list)):
    original_str = all_match_list[idx]
    new_str = all_match_list[idx][:-1]+';\n'
    code = code.replace(original_str, new_str)
all_match_list = re.findall('\)\n', code)
for idx in range(len(all_match_list)):
    original_str = all_match_list[idx]
    new_str = all_match_list[idx][:-1]+';\n'
    code = code.replace(original_str, new_str)

# handle missing values
all_match_list = re.findall('If.*Then', code)
for idx in range(len(all_match_list)):
    original_str = all_match_list[idx]
    new_str = ' '.join(original_str.split()[:-1] + ['and not missing', original_str.split()[1], ' Then'])
    code = code.replace(original_str, new_str)

# replace the 'inputVector' with var name
dictionary = {'inputVector(0)':'sepal_length',
              'inputVector(1)':'sepal_width',
              'inputVector(2)':'petal_length',
              'inputVector(3)':'petal_width'} 
for key in dictionary.keys():
    code = code.replace(key, dictionary[key])

# change the prediction labels
code = re.sub('Math.Exp', 'Exp', code)
code = re.sub('pred = .*\n', '', code)
temp_var_list = re.findall(r"var[0-9]+\(\d\)", code)
for var_idx in range(len(temp_var_list)):
    code = re.sub(re.sub('\\(', '\\(', re.sub('\\)', '\\)', temp_var_list[var_idx])), iris.target_names[var_idx]+'_prob', code)

In [11]:
print(code)

DATA pred_result;
SET dataset_name;    
    If (petal_length) < (2.45) and not missing (petal_length)  Then
        var0 = 0.41652894;
    Else
        If (petal_width) < (0.6) and not missing (petal_width)  Then
            var0 = 0.257142842;
        Else
            var0 = -0.209508225;
        
    
    
    If (petal_length) < (2.45) and not missing (petal_length)  Then
        var1 = 0.289331138;
    Else
        If (petal_width) < (0.6) and not missing (petal_width)  Then
            var1 = 0.211369321;
        Else
            var1 = -0.18690227;
        
    
    
    If (petal_length) < (2.45) and not missing (petal_length)  Then
        var2 = 0.231908903;
    Else
        If (petal_width) < (0.6) and not missing (petal_width)  Then
            var2 = 0.176793814;
        Else
            var2 = -0.170180246;
        
    
    
    If (petal_length) < (2.45) and not missing (petal_length)  Then
        var3 = 0.199418262;
    Else
        If (petal_width) < (0.6) and not mis

In [12]:
# save output
vb = open('vb2.sas', 'w')
vb.write(code)
vb.close()

In [13]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [14]:
col_names = ['sepal_length','sepal_width','petal_length','petal_width']

In [15]:
pd.DataFrame(X_test, columns = col_names).to_csv('iris_test2.csv',index=False)

## Compare prediction output

In [16]:
# python pred
python_pred = pd.DataFrame(model.predict_proba(X_test))
python_pred.columns = ['setosa_prob','versicolor_prob','virginica_prob']
python_pred

,setosa_prob,versicolor_prob,virginica_prob
0,0.001817,0.003775,0.994408
1,0.666191,0.321548,0.012262
2,0.049925,0.920066,0.030009
3,0.008012,0.986573,0.005414
4,0.008302,0.980696,0.011002
5,0.001481,0.984528,0.013991
6,0.000858,0.016383,0.982760
7,0.001981,0.997608,0.000412
8,0.985886,0.010111,0.004003
9,0.995535,0.003196,0.001269


In [19]:
# sas pred
sas_pred = pd.read_csv('pred_result2.csv')
sas_pred = sas_pred.iloc[:,-3:]
sas_pred

,setosa_prob,versicolor_prob,virginica_prob
0,0.001817,0.003775,0.994408
1,0.666191,0.321548,0.012262
2,0.049925,0.920066,0.030009
3,0.008012,0.986573,0.005414
4,0.008302,0.980696,0.011002
5,0.001481,0.984528,0.013991
6,0.000858,0.016383,0.982760
7,0.001981,0.997608,0.000412
8,0.985886,0.010111,0.004003
9,0.995535,0.003196,0.001269


In [20]:
(python_pred - sas_pred > 0.00001).sum()

setosa_prob        0
versicolor_prob    0
virginica_prob     0
dtype: int64